In [1]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/f2/8c/35364a11b3e25f8e29a35420b0d18f65ec4f9d6d38e86a62d16ef998923c/pandas-2.1.1-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.22.4 from https://files.pythonhosted.org/packages/2d/ed/022fc4106f6d97e41e156201274138e0369b27dbfc8c206034f24ebd97d9/numpy-1.26.1-cp39-cp39-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
    -----------------

In [2]:
!pip install ksql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/269.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/269.8 kB ? eta -:--:--
     -------------------------------------  266.2/269.8 kB 5.4 MB/s eta 0:00:01
     -------------------------------------- 269.8/269.8 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/71.9 kB ? eta -:--:--
     ---------------------------------------- 71.9/71.9 kB 3.9 MB/s eta 0:00:00
  Using cached hpack-3.0.0-py2.py3-none-any.whl (38 kB)
  Created wheel for ksql: filename=ksql-0.10.2-py3-none-any.whl size=12837 sha256=d6b1ea93e4abc0678c32744ddd2adc12533c3f6c7c6d7c191f4fc83db4c9b49c
  Stored in directory: c:\users\tjk\appdata\local\pip\cache\wheels\41\01\fe\a845eee067cabc26ab42264eb2c711f35400ceabfb44548757
Successfully built ksql


In [1]:
import logging
from ksql import KSQLAPI
import pandas as pd
import time
import datetime
import random

In [2]:
# 카프카에 저장된 데이터를 추가 조회 가능한 KSQLDB 접속 http://localhost:8089
# API 형식으로 카프카의 토픽에 데이터를 추가 조회 가능한 객체 리턴
client = KSQLAPI('http://localhost:8089', timeout=None)

In [3]:
# 카프카에 토픽(테이블) 조회
result = client.ksql('show topics;')
result

[{'@type': 'kafka_topics',
  'statementText': 'show topics;',
  'topics': [{'name': 'car_arrive_topic', 'replicaInfo': [1]},
   {'name': 'car_location_topic', 'replicaInfo': [1]},
   {'name': 'default_ksql_processing_log', 'replicaInfo': [1]}],
  'warnings': []}]

In [4]:
# 버스 위치 정보가 저장된 location.csv 읽기
location_df = pd.read_csv("location.csv")
location_df

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91769,302350,2019-11-05,405328102,7983486,281-2,33.493625,126.534764,제주지방법원(광양방면),20시,272.0,고산동산(광양방면),33.495540,126.532907
91770,302351,2019-11-05,405328102,7983486,281-2,33.495540,126.532907,고산동산(광양방면),20시,447.0,제주시청(광양방면),33.498925,126.530351
91771,302352,2019-11-05,405328102,7983486,281-2,33.498925,126.530351,제주시청(광양방면),20시,418.0,광양사거리,33.500473,126.527103
91772,302353,2019-11-05,405328102,7983486,281-2,33.500473,126.527103,광양사거리,20시,140.0,탐라장애인 종합복지관,33.500228,126.525625


In [5]:
location_df.sort_values(["date", "now_arrive_time"], inplace=True)
location_df

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
528,210985,2019-10-29,405136001,7997040,360-1,33.499560,126.529771,제주시청(아라방면),05시,454.0,고산동산(아라방면),33.496007,126.532287
529,210986,2019-10-29,405136001,7997040,360-1,33.496007,126.532287,고산동산(아라방면),05시,414.0,제주지방법원(아라방면),33.493109,126.535091
530,210987,2019-10-29,405136001,7997040,360-1,33.493109,126.535091,제주지방법원(아라방면),05시,525.0,제주중앙여자고등학교(아라방면),33.489414,126.538642
531,210988,2019-10-29,405136001,7997040,360-1,33.479705,126.543811,남국원(아라방면),05시,355.0,아라초등학교,33.476533,126.544283
532,210989,2019-10-29,405136001,7997040,360-1,33.456267,126.551750,제주대학교입구,05시,266.0,제대마을,33.457724,126.554014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91552,302133,2019-11-05,405328102,7983445,281-2,33.495540,126.532907,고산동산(광양방면),23시,447.0,제주시청(광양방면),33.498925,126.530351
91553,302134,2019-11-05,405328102,7983445,281-2,33.498925,126.530351,제주시청(광양방면),23시,418.0,광양사거리,33.500473,126.527103
91554,302135,2019-11-05,405328102,7983445,281-2,33.500473,126.527103,광양사거리,23시,140.0,탐라장애인 종합복지관,33.500228,126.525625
91555,302136,2019-11-05,405328102,7983445,281-2,33.500228,126.525625,탐라장애인 종합복지관,23시,165.0,홍랑로입구,33.499974,126.523865


In [6]:
location_df.reset_index(drop=True, inplace=True)
location_df

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210985,2019-10-29,405136001,7997040,360-1,33.499560,126.529771,제주시청(아라방면),05시,454.0,고산동산(아라방면),33.496007,126.532287
1,210986,2019-10-29,405136001,7997040,360-1,33.496007,126.532287,고산동산(아라방면),05시,414.0,제주지방법원(아라방면),33.493109,126.535091
2,210987,2019-10-29,405136001,7997040,360-1,33.493109,126.535091,제주지방법원(아라방면),05시,525.0,제주중앙여자고등학교(아라방면),33.489414,126.538642
3,210988,2019-10-29,405136001,7997040,360-1,33.479705,126.543811,남국원(아라방면),05시,355.0,아라초등학교,33.476533,126.544283
4,210989,2019-10-29,405136001,7997040,360-1,33.456267,126.551750,제주대학교입구,05시,266.0,제대마을,33.457724,126.554014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91769,302133,2019-11-05,405328102,7983445,281-2,33.495540,126.532907,고산동산(광양방면),23시,447.0,제주시청(광양방면),33.498925,126.530351
91770,302134,2019-11-05,405328102,7983445,281-2,33.498925,126.530351,제주시청(광양방면),23시,418.0,광양사거리,33.500473,126.527103
91771,302135,2019-11-05,405328102,7983445,281-2,33.500473,126.527103,광양사거리,23시,140.0,탐라장애인 종합복지관,33.500228,126.525625
91772,302136,2019-11-05,405328102,7983445,281-2,33.500228,126.525625,탐라장애인 종합복지관,23시,165.0,홍랑로입구,33.499974,126.523865


In [7]:
# 0번째 버스 위치 정보 조회
location_df.loc[0]

id                     210457
date               2019-10-29
route_id            405136001
vh_id                 7997025
route_nm                360-1
now_latitude        33.457724
now_longitude      126.554014
now_station              제대마을
now_arrive_time           07시
distance                333.0
next_station            제대아파트
next_latitude       33.458783
next_longitude     126.557353
Name: 0, dtype: object

In [8]:
# 카프카로 전송할 데이터의 수
count = len(location_df)
count

91774

In [9]:
# 인덱스의 시작
index = 0
# 무한 반복
while True:
    if index >= count:
        index = 0
    now = datetime.datetime.now()
    print("now =", now)
    
    date = now.strftime('%Y-%m-%d')
    print("date =", date)
    
    now_arrive_time = now.hour
    print("now_arrive_time =", now_arrive_time)
    
    weekday = now.weekday()
    print("weekday =", weekday)
    
    location_row = location_df.loc[index]
    
    route_id = location_df.loc[index, 'route_id']
    print("route_id =", route_id)
    
    vh_id = location_df.loc[index, 'vh_id']
    print("vh_id =", vh_id)
    
    route_nm = location_df.loc[index, 'route_nm']
    print("route_nm =", route_nm)
    
    now_latitude = round(location_df.loc[index, 'now_latitude'], 6)
    print("now_latitude =", now_latitude)
    
    now_longitude = round(location_df.loc[index, 'now_longitude'], 6)
    print("now_longitude =", now_longitude)
    
    now_station = location_df.loc[index, 'now_station']
    print("now_station =", now_station)
    
    distance = location_df.loc[index, 'distance']
    print("distance =", distance)
    
    next_latitude = round(location_df.loc[index, 'next_latitude'], 6)
    print("next_latitude =", next_latitude)
    
    next_longitude = round(location_df.loc[index, 'next_longitude'], 6)
    print("next_longitude =", next_longitude)
    
    next_station = location_df.loc[index, 'next_station']
    print("next_station =", next_station)
    
    # now.timestamp(): 현재 날짜와 시간 리턴
    # int로 현재 날짜와 시간을 정수로 변환
    id = int(now.timestamp())
    print("id =", id)
    
    insert_query = f"""INSERT INTO car_location_topic (
        id,
        route_id,
        vh_id,
        route_nm,
        now_latitude,
        now_longitude,
        now_station,
        now_arrive_time,
        distance,
        next_station,
        next_latitude,
        next_longitude,
        weekday,
        date
        ) VALUES ( {id}, '{route_id}', '{vh_id}',
        '{route_nm}', {now_latitude}, {now_longitude},
        '{now_station}', {now_arrive_time}, {distance},
        '{next_station}', {next_latitude}, {next_longitude},
        {weekday}, '{date}');"""
    
    print("insert_query =", insert_query)
    try:
        client.ksql(insert_query) # KSQLDB에 현재 위치 추가
    except Exception as e:
        print("예외가 발생했지만 데이터는 추가되었습니다: e =", e)
    
    sleep_time = random.randrange(120, 600)
    print("sleep_time =", sleep_time)
    time.sleep(sleep_time)
    
    # # 30초 대기
    # time.sleep(30)
    # 인덱스 1 증가
    index += 1
    print("=" * 100)

now = 2023-10-16 14:46:08.685325
date = 2023-10-16
now_arrive_time = 14
weekday = 0
route_id = 405136001
vh_id = 7997025
route_nm = 360-1
now_latitude = 33.457724
now_longitude = 126.554014
now_station = 제대마을
distance = 333.0
next_latitude = 33.458783
next_longitude = 126.557353
next_station = 제대아파트
id = 1697435168
insert_query = INSERT INTO car_location_topic (
        id,
        route_id,
        vh_id,
        route_nm,
        now_latitude,
        now_longitude,
        now_station,
        now_arrive_time,
        distance,
        next_station,
        next_latitude,
        next_longitude,
        weekday,
        date
        ) VALUES ( 1697435168, '405136001', '7997025',
        '360-1', 33.457724, 126.554014,
        '제대마을', 14, 333.0,
        '제대아파트', 33.458783, 126.557353,
        0, '2023-10-16');
예외가 발생했지만 데이터는 추가되었습니다: e = list index out of range
now = 2023-10-16 14:46:39.495068
date = 2023-10-16
now_arrive_time = 14
weekday = 0
route_id = 405136001
vh_id = 7997025
rou